---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
    
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    file = open('university_towns.txt')
    townAndState = file.readlines()
    #k=townList[1]
    townList = []
    for town in townAndState:
        #if 'Consortium' in town:
        #    continue
        #if 'Region' in town:
        #    continue
        if '(' in town:
            cutTown = town[0:(town.find('(')-1)]
        else:
            cutTown = town[0:-1]
        if 'edit' in cutTown: # [] indicates a state
            state = cutTown[0:cutTown.find('[')]
            continue
        #if ',' in cutTown:
        #    cutTown = cutTown[0:(cutTown.find(','))]
        townList.append([state, cutTown])
    answer = pd.DataFrame(townList, columns=["State", "RegionName"])
    return answer

"""df = get_list_of_university_towns()
cols = ["State", "RegionName"]

print('Shape test: ', "Passed" if df.shape ==
      (517, 2) else 'Failed')
print('Index test: '
      "Passed" if df.index.tolist() == list(range(517))
      else 'Failed')

print('Column test: ',
      "Passed" if df.columns.tolist() == cols else 'Failed')
print('\\n test: ',
      "Failed" if any(df[cols[0]].str.contains(
          '\n')) or any(df[cols[1]].str.contains('\n'))
      else 'Passed')
print('Trailing whitespace test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\s+$')) or any(df[cols[1]].str.contains(
          '\s+$'))
      else 'Passed')
print('"(" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\(')) or any(df[cols[1]].str.contains(
          '\('))
      else 'Passed')
print('"[" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\[')) or any(df[cols[1]].str.contains(
          '\]'))
      else 'Passed')
# from troubleshooting learners code, using incorrect regex
# for RegionName usually generates these common mismatches
rgn_rgx_mstch = [
    'Pomona', 'Mankato', 'Fulton', 'Sewanee']
rgn_loc = (33, 218, 237, 442)
print ('RegionName regex test: ', "Passed" if all(df.loc[
       rgn_loc, 'RegionName'] == rgn_rgx_mstch)
       else "Failed")
# when using split or find to extract regionName, these are
# common mismatches
rgn_splt_msmtch = [
    'The Five College Region of Western Massachusetts:',
    'Faribault, South Central College']
rgn_loc = (184, 217)
print ('RegionName regex test: ', "Passed" if all(df.loc[
       rgn_loc, 'RegionName'] == rgn_splt_msmtch)
       else "Failed")
rgn_loc = (184, 217)
print ('RegionName regex test: ', df.loc[
       rgn_loc, 'RegionName']) 
#test if . was  inadvertently removed
print('RegionName regex test:',"Passed" if any(df[cols[1]].str.contains(
          '\.')) else "Failed")
# use the values of the states dictionary precoded to verify
# state names are as expected
states_vlist = list(sorted(states.values()))
mismatchedStates = df[~df['State'].isin(
    states_vlist)].loc[:, 'State'].unique()
print ('State regex test: ', "Passed" if len(
    mismatchedStates) == 0 else "Failed")
if len(mismatchedStates) > 0:
    print()
    print('The following states failed the equality test:')
    print()
    print('\n'.join(mismatchedStates))
print_full(df)"""

'df = get_list_of_university_towns()\ncols = ["State", "RegionName"]\n\nprint(\'Shape test: \', "Passed" if df.shape ==\n      (517, 2) else \'Failed\')\nprint(\'Index test: \'\n      "Passed" if df.index.tolist() == list(range(517))\n      else \'Failed\')\n\nprint(\'Column test: \',\n      "Passed" if df.columns.tolist() == cols else \'Failed\')\nprint(\'\\n test: \',\n      "Failed" if any(df[cols[0]].str.contains(\n          \'\n\')) or any(df[cols[1]].str.contains(\'\n\'))\n      else \'Passed\')\nprint(\'Trailing whitespace test:\',\n      "Failed" if any(df[cols[0]].str.contains(\n          \'\\s+$\')) or any(df[cols[1]].str.contains(\n          \'\\s+$\'))\n      else \'Passed\')\nprint(\'"(" test:\',\n      "Failed" if any(df[cols[0]].str.contains(\n          \'\\(\')) or any(df[cols[1]].str.contains(\n          \'\\(\'))\n      else \'Passed\')\nprint(\'"[" test:\',\n      "Failed" if any(df[cols[0]].str.contains(\n          \'\\[\')) or any(df[cols[1]].str.contains(\n       

In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', header = 5, skiprows=range(6,220), parse_cols = "E:G")
    df['deltaGdp'] = df['GDP in billions of chained 2009 dollars'] - df['GDP in billions of chained 2009 dollars'].shift(1)
    df['negGdpChang'] = df['deltaGdp']<0
    df['recession'] = df['negGdpChang'] & df['negGdpChang'].shift(-1)
    answer = df[df['recession']==True].index[0]
    return answer




In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', header = 5, skiprows=range(6,220), parse_cols = "E:G")
    df['deltaGdp'] = df['GDP in billions of chained 2009 dollars'] - df['GDP in billions of chained 2009 dollars'].shift(1)
    df['negGdpChang'] = df['deltaGdp']<0
    df['recession end'] = ((df['negGdpChang'] == False) &  (df['negGdpChang'].shift(1) == False) &
                           (df['negGdpChang'].shift(2) == True) & (df['negGdpChang'].shift(3) == True))
    answer = df[df['recession end']==True].index[0] 
    return answer

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', header = 5, skiprows=range(6,220), parse_cols = "E:G")
    df['deltaGdp'] = df['GDP in billions of chained 2009 dollars'] - df['GDP in billions of chained 2009 dollars'].shift(1)
    df['negGdpChang'] = df['deltaGdp']<0
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    df2 = df.loc[recession_start:recession_end]  # isolates only the range of the recession
    answer = df2[df2['GDP in billions of chained 2009 dollars'] == min(df2['GDP in billions of chained 2009 dollars'] )].index[0]
    return answer

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    yrMos = df.columns.values[6:]
    df[yrMos[-3:]]
    df2 = df[['State', 'RegionName']]
    #np.mean(df[yrMos[0:3]],axis = 1)
    index = 0 
    for yrMo in yrMos:
        if int(yrMo[0:4]) < 2000:
            index = index+1
            continue 
        if '-01' in yrMo:
            yr = yrMo[0:4]
            quarter = 'q1'
            df2[yr+quarter] = np.mean(df[yrMos[index:(index+3)]],axis = 1)
        if '-04' in yrMo:
            yr = yrMo[0:4]
            quarter = 'q2'
            df2[yr+quarter] = np.mean(df[yrMos[index:(index+3)]],axis = 1)
        if '-07' in yrMo:
            yr = yrMo[0:4]
            quarter = 'q3'
            df2[yr+quarter] = np.mean(df[yrMos[index:(index+3)]],axis = 1)
        if '-10' in yrMo:
            yr = yrMo[0:4]
            quarter = 'q4'
            df2[yr+quarter] = np.mean(df[yrMos[index:(index+3)]],axis = 1)
        index = index+1
    df2 = df2.replace({'State': states})
    df2 = df2.set_index(['State', 'RegionName'])
    return df2

In [51]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    housingVals = convert_housing_data_to_quarters()
    uTowns = get_list_of_university_towns()
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    percent_change = (housingVals[recession_bottom] -housingVals[recession_start]) *100 / housingVals[recession_start]
    uTowns['is U town'] = True
    percent_change = pd.DataFrame(percent_change, columns= ['% change'])
    df2 = pd.merge(percent_change, uTowns, how='left', left_index=True, right_on=['State', 'RegionName'])
    df2['is U town'][df2['is U town'] != True] = False
    collegeTownChange = df2[df2['is U town'] == True]
    nonCollegeTownChange = df2[df2['is U town'] == False]
    collegeTownChange = collegeTownChange.dropna()
    nonCollegeTownChange = nonCollegeTownChange.dropna()
    statistic, pvalue = ttest_ind(collegeTownChange['% change'], nonCollegeTownChange['% change'])
    if statistic>0:
        better = 'university town'
    else:
        better = 'non-university town'
    if pvalue<0.01:
        different = True
    else:
        different = False
    answer = (different, pvalue, better)
    return answer



/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

2.61350012776
0.00897558754071


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
